In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, ShuffleSplit, cross_validate,cross_val_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import metrics, svm
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
import seaborn
from matplotlib import pyplot
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN
from sklearn.feature_selection import chi2, SelectKBest, SelectFdr
from sklearn.decomposition import PCA
from rake_nltk import Rake

### Preprocessing

Import data as data frame

In [2]:
problemfile = 'problems_2019-03-21.xlsx'
df = pd.read_excel(problemfile)
df

,Problem Manager,Number,Active,Activity due,Additional assignee list,Approval,Approval history,Approval set,Assigned to,Assignment group,...,Work notes,Work notes list,Workaround,Details,IT WAR Walk On,Quality Improvement Project,Director,Follow Up,Send Email,VP
0,Naveen Kumar,PRB0062159,True,NaN,NaN,Not Yet Requested,NaN,NaN,NaN,FSC_ITL3O2STIBCO,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,None,NaN
1,Mark Duncan,PRB0060146,False,NaN,NaN,Not Yet Requested,NaN,NaN,V Sox,FXF_SPT_US_FIELDLINEHAUL,...,2018-08-22 05:40:24 - Eli Smith (Work notes)\n...,NaN,2018-06-28 19:56:05 - D'Zundra Green (Workarou...,NaN,NaT,NaT,NaN,NaN,None,NaN
2,Naveen Kumar,PRB0060758,False,NaN,NaN,Not Yet Requested,NaN,NaN,NaN,FSC_ITL3O2STIBCO,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,None,NaN
3,Christophe Gurley,PRB0060985,False,NaN,NaN,Not Yet Requested,NaN,NaN,Scott Dubak,FXO_SA_US_WindowsServer,...,2018-08-12 14:49:21 - Christophe Gurley (Work ...,NaN,2018-07-15 18:14:43 - Ashish Bisht (Workaround...,NaN,NaT,NaT,NaN,NaN,None,NaN
4,Michael Kennemer,PRB0060147,False,NaN,NaN,Not Yet Requested,NaN,NaN,Robert Bumpus,FXS_FIREWALL,...,2018-07-27 15:09:39 - Michael Kennemer (Work n...,NaN,2018-06-29 07:24:25 - Brad Moore (Workaround)\...,NaN,NaT,NaT,NaN,NaN,None,NaN
5,Michael Kennemer,PRB0062412,True,NaN,NaN,Not Yet Requested,NaN,NaN,Mahesh Pillutla,FXF_SPT_US_FreightDataSvcs,...,NaN,NaN,2019-01-06 12:00:08 - Stephen Barch (Workaroun...,Connection pooling errors,NaT,NaT,Rita Moore,NaN,None,Ann Higgins
6,Christopher Barber,PRB0063553,False,NaN,NaN,Not Yet Requested,NaN,NaN,NaN,FXS_EIS_GLB_ProblemMgmt,...,NaN,NaN,NaN,NaN,NaT,NaT,NaN,NaN,None,NaN
7,Terri Hamilton,PRB0060194,False,NaN,NaN,Not Yet Requested,NaN,NaN,William Redmond,FXS_SPT_US_DOTCOMSHIPPING,...,2018-09-13 09:57:53 - Michael Kennemer (Work n...,NaN,2018-07-06 20:12:25 - Sean Green (Workaround)\...,NaN,NaT,NaT,NaN,NaN,None,NaN
8,Eli Smith,PRB0061073,False,NaN,NaN,Not Yet Requested,NaN,NaN,Douglas Clinger,FXF_SPT_SEFS,...,2018-07-26 03:00:28 - Eli Smith (Work notes)\n...,NaN,2018-07-24 03:21:28 - Eli Smith (Workaround)\n...,NaN,NaT,NaT,NaN,NaN,None,NaN
9,Eli Smith,PRB0061373,False,NaN,NaN,Not Yet Requested,NaN,NaN,Matthew Schwab,FXS_SPT_GLB_SharePoint,...,2018-12-11 14:41:16 - Randall Painter (Work no...,NaN,2018-09-15 07:26:49 - Divakar Durgapal (Workar...,NaN,NaT,NaT,NaN,NaN,None,NaN


In [3]:
data = df.dropna(axis=1, how='all')
data = df.dropna(axis=1, thresh=df.shape[0]*0.90)
for col in data:
    if len(data[col].value_counts()) < 2:
        data = data.drop(col, axis=1)
data = data.drop(['Active', 'State'], axis=1)
data['Parent'] = df['Parent']
data.to_csv('problem_pdsm_simple.csv', index=False)

In [7]:
data = data.reindex(sorted(data.columns), axis=1)
data.columns

Index(['Assignment group', 'Business duration', 'Business service', 'Created',
       'Created by', 'Description', 'Duration', 'Impact', 'Impacted OpCos',
       'Known error', 'Major Problem', 'Number', 'Opened', 'Opened by',
       'Parent', 'Priority', 'Problem Manager', 'Problem state',
       'Reassignment count', 'Related Incidents', 'Short description', 'Type',
       'Updated', 'Updated by', 'Updates', 'Urgency'],
      dtype='object')

### Extract keywords from descriptions in full dataset
Adds Short Description and Keywords features to above dataset

In [4]:
def extract_keywords(text):
    r = Rake(min_length=2, max_length=8)
    r.extract_keywords_from_text(text)
    return r.get_ranked_phrases()[0:4]

In [ ]:
data['Keywords'] = data['Short description'].apply(extract_keywordsct_keywords)

In [ ]:
# full_data = pd.read_excel('problems_2019-03-21.xlsx')

# desc_data = full_data[['Number', 'Short description']]
# desc_data = desc_data[desc_data['Number'].isin(data['Number'])]
# data = data.merge(desc_data, how='left', on='Number')
# data['Keywords'] = data['Short description'].apply(extract_keywords)
# data.head()

### Get number of incidents associated with each problem

In [ ]:
incidents = pd.read_excel('incidents_2019-03-21.xlsx')
incidents.head()

In [ ]:
# prob_counts = incidents['Problem'].value_counts().reset_index()
# prob_counts = prob_counts.rename(columns={'index': 'Number', 'Problem': 'Incidents Count'})
# prob_counts.head()

In [ ]:
# data = data.merge(prob_counts, how='left', on='Number')
# data['Incidents Count'] = data['Incidents Count'].fillna(0).astype(int)
# data

### Add assignment group from incidents to problems dataset

In [ ]:
# assign_groups = incidents[['Problem', 'Assignment group']]
# assign_groups = assign_groups.rename(columns={'Problem':'Number'}).drop_duplicates('Number')
# # assign_groups.head()
# data = data.merge(assign_groups, how='left', on='Number')
# data.head()

### Get avg, min, and max duration of related incidents for each problem

In [ ]:
durations = incidents[['Problem','Duration']]
durations = durations.rename(columns={'Problem':'Number'})
durations2 = data[['Number', 'Duration']]
durations = durations.merge(durations2, how='left', on='Number')
durations

In [ ]:
# durations.groupby(['Number']).mean()
durations['Duration Max'] = durations.Number.map(durations.groupby(['Number'])['Duration'].max())
durations['Duration Mean'] = durations.Number.map(durations.groupby(['Number'])['Duration'].mean())
durations['Duration Min'] = durations.Number.map(durations.groupby(['Number'])['Duration'].min())
durations['Duration Range'] = durations['Duration Max'] - durations['Duration Min']
durations = durations.drop_duplicates('Number').drop('Duration', axis=1)
durations.head()

In [ ]:
durations2 = data[['Number', 'Duration']]

In [ ]:
data = data.merge(durations, how='left', on='Number')
data.head()

### Find similar keywords within related incidents

## Clean the data

Find out default types for the columns

In [ ]:
data.dtypes

### Number column

Remove the PRB prefix from the Number column and convert it to a number

In [ ]:
data['Number'] = data['Number'].map(lambda x: x.lstrip('PRB'))
data['Number'] = pd.to_numeric(data['Number'])
data

In [ ]:
data.dtypes

### Priority, Impact, and Urgency

Take the first character (number) from each and convert to numeric

In [ ]:
data['Priority'] = data['Priority'].map(lambda x: x[0])
data['Priority'] = pd.to_numeric(data['Priority'])

data['Impact'] = data['Impact'].map(lambda x: x[0])
data['Impact'] = pd.to_numeric(data['Impact'])

data['Urgency'] = data['Urgency'].map(lambda x: x[0])
data['Urgency'] = pd.to_numeric(data['Urgency'])

data.head()

In [ ]:
data.dtypes

### Categorical Fields

Check out category counts

In [ ]:
data['Business service'].value_counts()

In [ ]:
data['Category'].value_counts()

In [ ]:
data['Company'].value_counts()

In [ ]:
data['Problem Manager'].value_counts()

In [ ]:
data['Type'].value_counts()

In [ ]:
data['Assignment group'].value_counts()

Cut categories with low counts and add together to make an "Other" category

In [ ]:
data = data.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name=='Business service' else x)
data = data.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name=='Company' else x)
data = data.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name=='Problem Manager' else x)
data = data.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name=='Type' else x)
data = data.apply(lambda x: x.mask(x.map(x.value_counts())<10, 'Other') if x.name=='Assignment group' else x)
data.head()

Convert categorical fields to be a category type

In [ ]:
data['Business service'] = data['Business service'].astype('category')
data['Category'] = data['Category'].astype('category')
data['Company'] = data['Company'].astype('category')
data['Problem Manager'] = data['Problem Manager'].astype('category')
data['Type'] = data['Type'].astype('category')
data['Assignment group'] = data['Assignment group'].astype('category')
data.head()

In [ ]:
data.dtypes

### Created

Convert the Created column to a datetime type

In [ ]:
data['Created'] = data['Created'].astype('datetime64[ns]')
data.head()

In [ ]:
data.dtypes

### Created by

Convert the Created by column to a number

In [ ]:
data['Created by'] = pd.to_numeric(data['Created by'])
data.head()

Looks like there may be some that have "admin" before the id number. I am just going to remove this prefix.

In [ ]:
data['Created by'] = data['Created by'].map(lambda x: x.lstrip('admin'))
data['Created by'] = pd.to_numeric(data['Created by'])
data.head()

In [ ]:
data.dtypes

### Impacted OpCos

Split Impacted OpCos column into list of OpCos instead of string

In [ ]:
data['Impacted OpCos'] = data['Impacted OpCos'].map(lambda x: x.split(","))

Weird.. Let's look for that float 

In [ ]:
data[data['Impacted OpCos'].apply(np.isreal)]

Looks like the NaNs are the problem. Convert NaNs to empty strings.

In [ ]:
data['Impacted OpCos'] = data['Impacted OpCos'].replace(np.nan, '', regex=True)
data[data['Impacted OpCos'].apply(np.isreal)]

There's no longer any NaNs, so let's try the split again

In [ ]:
data['Impacted OpCos'] = data['Impacted OpCos'].map(lambda x: x.split(","))
data.head()

Now we have the list, but we can't do anything with the list. Let's convert to dummy variables.

In [ ]:
pd.get_dummies(data['Impacted OpCos'].apply(pd.Series).stack()).sum(level=0).head()

That will take a lot of further processing, so I am going to drop that for now.

In [ ]:
data = data.drop('Impacted OpCos', axis=1)
data.head()

In [ ]:
data.dtypes

### Parent

Convert Parent column to a boolean Has Parent column

In [ ]:
data['Has Parent'] = data['Parent'].map(lambda x: not(np.isreal(x)))
data = data.drop('Parent', axis=1)
data.head()

In [ ]:
data.dtypes

Now all columns have a proper data type; let's drop the NaNs and look at our clean data set

In [ ]:
data = data.dropna()
data

### Adding categorical time of day feature

In [ ]:
from datetime import *
def convert_datetime(ts):
    h = ts.hour
    return ('morning' if 5 <= h < 12
        else 'afternoon' if 12 <= h < 17
        else 'evening' if 17 <= h < 21
        else 'night')

data.insert(loc=6, column='Created Time', value=data['Created'].apply(convert_datetime))
data['Created Time'] = data['Created Time'].astype('category')
data.head()

### Export most recently cleaned data to csv

In [ ]:
data.to_csv('problems_cleaned.csv', index=False)

### Clean up dataset with NAs

In [ ]:
prob_data = pd.read_csv('problems_cleaned_withna.csv')
prob_data

In [ ]:
prob_data.isna()